# Código para abrir cada um dos sitemaps.xml

In [1]:
# BEGIN IMPORTS
#xml to pandas example from http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/
import xml.etree.ElementTree as ET
import pandas as pd
from urllib.request import urlopen
# END IMPORTS

def abrir_maps(lista_urls):
    def xml2df(url_baixar):
        xml_data = urlopen(url_baixar).read()
        root = ET.XML(xml_data) # element tree
        all_records = [] #This is our record list which we will convert into a dataframe
        for i, child in enumerate(root): #Begin looping through our root tree
            record = {} #Place holder for our record
            for subchild in child: #iterate through the subchildren to user-agent, Ex: ID, String, Description.
                record[subchild.tag] = subchild.text #Extract the text create a new dictionary key, value pair
                all_records.append(record) #Append this record to all_records.
        return pd.DataFrame(all_records) #return records as DataFrame
    df = pd.DataFrame()
    
    for url in lista_urls:
        print(url)
        aux = xml2df(url)
        aux.columns = ['loc', 'lastmod']
        aux['lastmod'] = aux['lastmod'].astype('datetime64[ns]')
        df = df.append(aux)
    return df

In [ ]:
df_sitemap = pd.read_csv( 'LexMLSiteMap_07-02-2020.csv', encoding='utf-8' )
df_sitemap.info()

In [ ]:
df_urn_lexml = abrir_maps( df_sitemap['loc'] )
df_urn_lexml.info()
df_urn_lexml.drop_duplicates(inplace=True)
#df_urn_lexml.to_csv("LexML_URNs_07-02-2020.csv")
df_urn_lexml.info()

In [ ]:
#nao da pra salvar com excel pq a ehmuito grande pra salvar
#df_urn_lexml.to_excel("LexML_URNs_07-02-2020.xls", encoding='utf-8', index=False )

## Carregamento de todo o conteúdo dos sitemaps consolidados em um arquivo já salvo
<br> Assim não precisa ficar lendo todos os arquivos toda vez que for trabalhar neste notebook.

In [2]:
df_urn_lexml = pd.read_csv( 'LexML_URNs_07-02-2020.csv', encoding='utf-8' )
df_urn_lexml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8353496 entries, 0 to 8353495
Data columns (total 3 columns):
Unnamed: 0    int64
loc           object
lastmod       object
dtypes: int64(1), object(2)
memory usage: 191.2+ MB


### Seleciona apenas o TRT3

In [3]:
selecao = "regiao.3:tribunal.regional.trabalho;"
df_urn_trt3 = df_urn_lexml[ df_urn_lexml[ 'loc' ].str.contains( selecao ) ] 
df_urn_trt3.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1006586 entries, 3723133 to 8300261
Data columns (total 3 columns):
Unnamed: 0    1006586 non-null int64
loc           1006586 non-null object
lastmod       1006586 non-null object
dtypes: int64(1), object(2)
memory usage: 30.7+ MB


### Seleciona apenas os anos de 2017, 2018 e 2019

In [4]:
selecao_acordao_2017 = df_urn_trt3[ 'loc' ].str.contains( "acordao:2017" )
selecao_acordao_2018 = df_urn_trt3[ 'loc' ].str.contains( "acordao:2018" )
selecao_acordao_2019 = df_urn_trt3[ 'loc' ].str.contains( "acordao:2019" )

df_urn_trt3_selecionadas = df_urn_trt3[ selecao_acordao_2017 | selecao_acordao_2018 | selecao_acordao_2019 ]
df_urn_trt3_selecionadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268691 entries, 3813957 to 8300261
Data columns (total 3 columns):
Unnamed: 0    268691 non-null int64
loc           268691 non-null object
lastmod       268691 non-null object
dtypes: int64(1), object(2)
memory usage: 8.2+ MB


### Cálculo do tamanho da amostra
<br> Para estar certo da representatividade de decisões judiciais publicadas em 99% das vezes e que a amostra estatística extraída esteja no máximo 1% abaixo ou 1% acima da média da população, a quantidade deve ser de no mínimo 15.671. Optou-se por extrair quantidade maior para haver observações sobressalente no caso de algumas serem necessárias ser excluídas em fases posteriores de processamento.

In [5]:
df_urn_trt3_selecionadas=df_urn_trt3_selecionadas.drop( df_urn_trt3_selecionadas.columns[0],1)
df_urn_trt3_selecionadas.sample( 30000, random_state=1 ).info()
df_urn_trt3_selecionadas.sample( 30000, random_state=1  ).to_csv("TRT3_df_urn_selecionadas_amostra_2017_2018_2019.csv", encoding='utf-8', index=False )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 4407679 to 7827408
Data columns (total 2 columns):
loc        30000 non-null object
lastmod    30000 non-null object
dtypes: object(2)
memory usage: 703.1+ KB


In [5]:
df_urn_trt3_selecionadas=df_urn_trt3_selecionadas.drop( df_urn_trt3_selecionadas.columns[0],1)
df_urn_trt3_selecionadas.sample( 10000, random_state=2 ).info()
df_urn_trt3_selecionadas.sample( 10000, random_state=2  ).to_csv("TRT3_df_urn_selecionadas_amostra_2017_2018_2019_2.csv", encoding='utf-8', index=False )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 7793855 to 4395662
Data columns (total 2 columns):
loc        10000 non-null object
lastmod    10000 non-null object
dtypes: object(2)
memory usage: 234.4+ KB
